The aim of this code is to both understand group theory and create a framework for exploring group properties.

Before constructing the group classes I need to introduce a set wrapper class that adds some usefull functionality and renames some of the default set functions.

In [1]:
class Set(set):
    def __init__(self, elements):
        if type(elements) != set and isinstance(elements, set):
            elements = elements.elements
        self.set_elements(elements)
        self.universe = self.elements.copy()
    def __str__(self):
        return(self.elements.__str__())
    def __iter__(self):
        return self.elements.__iter__()
    def __len__(self):
        return self.elements.__len__()
    def __eq__(self, other):
        if type(other) == list:
            other = set(other)
        if type(other) == set:
            return self.elements == other
        elif type(other) == type(self):
            print(type(self))
            return self.elements == other.elements
        else:
            print("= error")
    def set_elements(self, elements):#
        self.elements = set(elements.copy())
    def share_universe(self, other):#adds elements from another and syncs universe
        self.universe.update(other.elements)
        other.__set_universe(self.universe)
    def __set_universe(self, u):#assigns the same set, not a copy
        self.universe = u
    def same_universe(self, other):
        return self.universe == other.universe
    def add(self, *args):#eqivalent to .add() set method, but optimized for the class
        for a in args:
            if type(a) == type(self):
                self.elements.update(a.elements)
            elif type(a) == set:
                self.elements.update(a)
            else:
                self.elements.add(a)
    def remove(self, e):
        self.elements.remove(e)
    def build_subset(self, predicate):#returns a filtered set, which is a subset 
        #should add a map function later
        #may add functionality where a subset is created with its universe set to the one that generates it
        return Set([x for x in self.elements if predicate(x)])
    def subset_of(self, other):#.issubset() 
        return len([x for x in self if x in other]) == len(self)
    def union(self, *args):#.union() python | operate or .update() with modification
        combination = self.elements.copy()
        for a in args:
            combination.update(a)
        return Set(combination)
    def __or__(self, other):
        return self.union(other)
    def intersect(self, *args):#.intersection() in python
        sect = None
        for a in args:
            sect = self.build_subset(lambda e : e in a)
        return sect
    def __and__(self, other):
        return self.intersect(other)
    def disjoint(self, other):#.isdisjoint() in python
        return self.intersect(other) == set()#empty set
    def subtract(self, other):#difference
        return self.build_subset(lambda e: e not in other)
    def complement(self):
        return Set(self.universe).subtract(self)
    def __p(self, s):#got from some wiki don't remember where
        s = list(s)
        if s==[]: # base case
            return [s] # if s is empty, then the only sublist of s is s itself
        else:
            e = s[0] # any e from s (in this implementation, we choose the first e)
            t = s[1:] # s with e removed
            pt = self.__p(t) # the list of all sublists of t (note that this is a recursive call)
            fept = [x + [e] for x in pt] # pt with e appended to each sublist
            return pt + fept # the concatenation of all constructed sublists
    def power_set(self):#returns a list of sets otherwise have to deal with frozensets witch can't be check directly with in frozenset
        return [x for x in self.__p(self.elements)]
    def set_product(self, other):
        product = set()
        for x in self.elements():
            for y in self.elements():
                product.add((x, y))
        return product
    

In [2]:
A = Set({1,2,3})
B = Set({3,1,1,4})
print("universe test:")
print("universe of A:", A.universe)
B.share_universe(A)
print("universe of A or B:", A.universe)

universe test:
universe of A: {1, 2, 3}
universe of A or B: {1, 2, 3, 4}


Now I'm writing the classes for and related to groups, I may retroactively add functionality to the set class to generate the following under a given operation.

In [22]:
#generalized associativity applies as well as generalized commutativity if abelian
#if a is an element of a semigroup, then a^n is the standard product of the binary function applied n times
#note to self: create permutation elements
class Semigroup(Set):#while inherits from sets, should overide adding elements as Semigroups are immutable
    def __init__(self, bfun, elements):
        super().__init__(elements)
        if not self.issemigroup(bfun, elements):
            print("error: not a semigroup")
        self.bfun = bfun #binary function that operates on elements
        self.order = len(elements)
    @staticmethod
    def issemigroup(bfun, elements):
        #to check associativity, it is necessary to brute force a(bc) = (ab)c given a,b,c are in elements
        e = list(elements)
        mod = len(elements)
        for a in elements:
            for b in elements:
                for c in elements:
                    if a == b and b == c:#assuming equality (__eq__) is defined
                        continue
                    #print(a,b,c, bfun(a, bfun(b, c)), bfun(bfun(a, b), c), bfun(a, bfun(b, c)) == bfun(bfun(a, b), c))
                    if bfun(a, bfun(b, c)) != bfun(bfun(a, b), c):
                        return False
        return True
    @staticmethod
    def isabelian(bfun, elements):#checks for commutative property ab = ba for all a,b in elements
        for a in elements:
            for b in elements:
                if bfun(a,b) != bfun(b,a):
                    return False
        return True
    def ishomomorphicto(self, ufun, H):#checks unary homomorphism f:G -> such that f(a)fa(b) = f(ab) for all a,b in G(self)
        #need to check that f:G->H will implement soon: injective -> monomorphism, surjective ->epimorphism
        #bijetive -> isomorphism, homomorphic f: G -> G is endomorpism, isomorphism f:G -> G is automorphism
        for a in self:
            for b in self:#H.bfun since the group being mapped to may have a different binary function
                if not H.bfun(ufun(a), ufun(b)) == ufun(self.bfun(a, b)):#can optimize if abelian
                    return False
        return True
    def isinjectiveto(self, ufun, H):#one to one, easy to show false if order, cardinality,size of self > order of other group
        ones = []#elements mapped to H
        for one in self:
            num = ufun(one)
            if num in ones:#checks for instance of mapping elements to the same output
                return False
            else:
                ones.append(num)
        return True
    def ismonomorphicto(self, ufun, H):# -> ker f = {e_H}
        return self.ishomomorphicto(ufun,H) and self.isinjectiveto(ufun, H)
    def issurjectiveto(self, ufun, H):#onto, for h in H, there exists g in G(self) such that f(g) = h
        #G is domain f: G -> H, the image of G == Im f surjective -> H is subset F(G)
        #easy to show false if there are not enough elements in G to cover H during the check
        s = set()
        for g in self:
            num = ufun(g)
            if num in H:
                s.add(ufun(g))
        if len(s) == H.order:
            return True
        else:
            False
    def isepimorphicto(self, ufun, H):
        return self.ishomomorphicto(ufun,H) and self.issurjectiveto(ufun, H)
    def isbijectiveto(self, ufun, H):
        return isinjectiveto(ufun, H) and Issurjectiveto(ufun, H)
    def isisomorphicto(self, ufun, H):#iff there exists an inverse function and the compositions ff^-1 = 1_G, f^-1f = 1_H
        #automorphism if f:G -> G
        return ishomomorphicto(ufun, H) and isbijectiveto(ufun, H)

In [4]:
addfun = lambda a, b : a + b
print(addfun(0,0))
stuff = {0,1,2,4,7,15}
print("addfun", Semigroup.issemigroup(addfun, stuff))
powfun = lambda a, b : a ** b
print(powfun(2,3))
print("powfun", Semigroup.issemigroup(powfun, stuff))
nums = Semigroup(addfun, stuff)
nums

0
addfun True
8
powfun False


{0, 1, 2, 4, 7, 15}

Identity in Monoid is unique
a^0 = e

In [5]:
class Monoid(Semigroup):
    def __init__(self, bfun, elements):#check if set is a semigroup with 2 sided identity
        super().__init__(bfun, elements)
        self.e = self.identity = self.find_identity()
        if self.identity == None:
            print("error: not a monoid")
            
    def find_identity(self):#checks for 2 sided identity
        for e in self.elements:
            if len([a for a in self.elements if self.bfun(a,e) == a and self.bfun(e,a) == a]) == len(self.elements):
                return e
        return None
            

In [6]:
Monoid(addfun, nums)

{0, 1, 2, 4, 7, 15}

A group G has,
if c is in and cc = c, then c = e the identity.
cancelation
unique inverses
(a^-1)^-1 = a
(ab)^-1 = b^-1a^-1
ax = b and ya = b have unique solutions x = a^-1b and ba^-1
a^-n = (a^-1)^n

a^ma^n = a^(m+n)
(a^m)^n = a^mn
in additive notation respectively
ma + na = (m + n)a
n(ma) = mna

homomorphism of groups have:
f:G -> H
f(e_G) = e_H
also f(a^-1) = f(a)^-1

In [20]:
class Group(Monoid):
    def __init__(self, bfun, elements):
        super().__init__(bfun, elements)
        self.inverses = self.find_inverses()
        if len(self.inverses) != len(elements):
            print(len(self.inverses), len(elements))
            print("error:not a group")
    def find_inverses(self):#checks a^-1a = aa^1 = e for all a in elements, 2 sided inverse
        container = self.elements.copy()
        container.remove(self.e)
        inverses = {self.e:self.e}#dictionary starting with e
        for a in container:
            for b in container:
                if b in inverses:
                    continue
                #print(self.bfun(a,b),self.bfun(b,a))
                if self.bfun(a,b) == self.e and  self.bfun(b,a) == self.e:
                    #print(self.bfun(a,b),self.bfun(b,a))
                    inverses[a] = b
                    if a != b:
                        inverses[b] = a
        return inverses
    def kernel(self, ufun, H): #unary f:G->H
        return self.buildsubset(lambda g: ufun(g) == H.e)#returns subset filtered by f(g) == e
    def idealof(self, other):'''ideal if any element in ideal in a binary function with an element in the group 
        results in an element in the ideal'''#work on later for rings
        pass
    def isSubgroupof():#under the same binary function, instantiating an instance comfirms a subset traditionally
        #closure also suffics, not necessary rn
        pass

In [14]:
stuff = {0,1,2,4,7,15}
multfun = lambda a,b: a*b
#not_group = Group(addfun, stuff)
#print(not_group)
#stuff.update([1/x if x !=0 else 0 for x in stuff])
stuff.update([-x if x !=0 else 0 for x in stuff])
import decimal
#decimal.getcontext().prec = 1000#would probably have to use fractions or rationals to wotrk with multiplication groups
#stuff = set([decimal.Decimal(x)for x in stuff])
#print(stuff)
g = Group(addfun, stuff)
print(g)

{0, 1, 2, 4, 7, 15, -15, -2, -7, -4, -1}


I plan to add both equivalence relations and groups of permuations